# NOSQL
`NOSQL` (key-value) is a an alternative database solution to `SQL` (relational databases). 

It uses a collection - document model as opposed to highly structured manner in which SQL is implemented. Why would you consider NOSQL, here are a few advantages:

- Massive potential to scale horizontally
- Open source
- Fosters quick development, easy to learn and use
- No need to deal with complex joins
- Freedom to structure your data how you want
- Requires little to no data massaging (data is inserted/queried as arrays/dictionaries)

I will be using a popular open source NOSQL database called **MongoDB** in this notebook.

### Installation (optional)
If you have a MongoDB database available in the cloud you can skip this step. An instance of MongoDB has been installed to my local Ubuntu 19.04 VM and the commands for doing so are below. If you're on Mac you will most likely need to use brew to install MongoDB.
```sh
sudo apt upgrade
sudo apt install -y mongodb
```

You will also need to install `pymongo` in order to connect to MongoDB from python. After you've activated your virtual environment. In addition, you can also access the database from MongoDB's shell.
```sh
pip install pymongo
```

### Imports 

In [1]:
import pymongo

### Connecting

In [2]:
uri = 'mongodb://localhost:27017/'

connection = pymongo.MongoClient(uri)

### Verifying connection 
Calling `server_info()` function on the connection object shows all sorts of info about the database and connection.

In [10]:
connection.server_info()

{'version': '3.6.8',
 'gitVersion': '8e540c0b6db93ce994cc548f000900bdc740f80a',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [3, 6, 8, 0],
 'openssl': {'running': 'OpenSSL 1.1.1b  26 Feb 2019',
  'compiled': 'OpenSSL 1.1.1a  20 Nov 2018'},
 'buildEnvironment': {'distmod': '',
  'distarch': 'x86_64',
  'cc': 'cc: cc (Ubuntu 8.2.0-12ubuntu1) 8.2.0',
  'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Wno-error=c++1z-compat -Wno-error=noexcept-type -Wno-error=format-truncation -Wno-error=int-in-bool-context -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -Wno-format-truncation -fstack-protector-strong -fno-builtin-memcmp',
  'cxx': 'g++: g++ (Ubuntu 8.2.0-12ubuntu1) 8.2.0',
  'cxxflags': '-g -O2 -fdebug-prefix-map=/build/mongod

### Listing Databases
Databases can be listed with the `list_databases()` function. Note that these are the default databases coming with the initial installation of MongoDB.

In [13]:
for db in connection.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 32768.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 12288.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 32768.0, 'empty': False}


### Creating a database 
Unlike `SQL` where your schema creates the tables beforehand. In NOSQL the database and collections arent created until data is inserted into them. For now Ill make a database called 'office'

In [14]:
database = connection['office']

### Creating a collection
Within the office database ill create a furniture collection

In [16]:
furniture = database['furniture']

### Inserting single document into a collection
Within the furniture collection I can insert one document in the form of a python dictionary, using `insert_one`

In [18]:
garys_chair = {
                'brand':'furniture r us',
                'current-owner':'Gary Goods',
                'type':'leather',
                'weight-cap-kg': 150,
                'colour': 'green',
                'purchased': '1990-12-20',
                'cleaned': ['1991-05-01', '2001-05-05']
              }

In [19]:
furniture.insert_one(garys_chair)

### Inserting multiple documents into a collection
Alternatively you can insert many dictionaries in a list, by calling `insert_many` on your collection object

In [27]:
tables = [{'brand':'furniture r us', 'length': 180, 'width': 30, 'area': 'kitchen'},
          {'brand':'furniture r us', 'length': 180, 'width': 30, 'area': 'meeting room 1'},
          {'brand':'furniture r us', 'length': 180, 'width': 30, 'area': 'meeting room 2'}]

In [28]:
furniture.insert_many(tables)

### Retrieving documents
Retrieve all documents from collection like so, you can also pass in filter dictionary. For example {'colour':'red'} will return only documents matching that criteria.

In [29]:
results = furniture.find()

In [30]:
for x in results:
    print(x)

{'_id': ObjectId('5cee07433041c7553e0fcc51'), 'brand': 'furniture r us', 'current-owner': 'Gary Goods', 'type': 'leather', 'weight-cap-kg': 150, 'colour': 'green', 'purchased': '1990-12-20', 'cleaned': ['1991-05-01', '2001-05-05']}
{'_id': ObjectId('5cee080d3041c7553e0fcc52'), 'brand': 'furniture r us', 'length': 180, 'width': 30, 'area': 'kitchen'}
{'_id': ObjectId('5cee080d3041c7553e0fcc53'), 'brand': 'furniture r us', 'length': 180, 'width': 30, 'area': 'meeting room 1'}
{'_id': ObjectId('5cee080d3041c7553e0fcc54'), 'brand': 'furniture r us', 'length': 180, 'width': 30, 'area': 'meeting room 2'}


### Updating a document
Updating is fairly straight forward too. You use `$set` command. There are more commands like this, for example `$push` can be used to insert more objects into an array within a document. In this example I change the document where Gary Goods is the owner from colour green to red, and print the results afterwards

In [31]:
furniture.update_one({'current-owner': 'Gary Goods'}, {'$set':{'colour': 'red'}})

In [32]:
furniture.find({'current-owner':'Gary Goods'})[0]

{'_id': ObjectId('5cee07433041c7553e0fcc51'),
 'brand': 'furniture r us',
 'current-owner': 'Gary Goods',
 'type': 'leather',
 'weight-cap-kg': 150,
 'colour': 'red',
 'purchased': '1990-12-20',
 'cleaned': ['1991-05-01', '2001-05-05']}

### Deleting a document 
If we want to remove a document, `delete_one` can be called on the collection object, specifying a criteria for deletion.

In [33]:
furniture.delete_one({'area': 'meeting room 2'})